<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="25%"><img src="media/decartes.jpg"
alt="DeCART Icon" width="128" height="171"><br>
</td>
<td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">DeCART Summer School<br>
for<br>
Biomedical Data Science</font></h1></td>
<td valign="center" align="center" width="25%"><img
src="media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


# Working with Textual Data
#### &copy; Brian E. Chapman, Ph.D.


### To get the latest version of this lesson, execute the cell below

In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob

In [3]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [4]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [10]:
rad_Data=pd.read_sql("""SELECT * 
               FROM noteevents
               """,conn)
rad_Data.head()
set(rad_Data["category"])

{'DISCHARGE_SUMMARY', 'MD Notes', 'Nursing/Other', 'RADIOLOGY_REPORT'}

In [5]:
rad_data.shape

(10000, 3)

## Section Splitting

<img src="http://medicine.utah.edu/dbmi/images/faculty/Chapman,Wendy_240x360.jpg"
alt="Wendy Chapman" width="128">

Clinical reports are often divided into sections. Physicians are taught to write their reports in the SOAP format: subjective information, objective information, assessment, and plan, and sections often reflect this process. Information each setion may be formatted differently. For example, subjective information is often narrative, telling a story of the patient's description of their past medical history and their current issues. The objective section may be much more telegraphic with shortened descriptions of measurements like blood pressure and heart rate, such as "120/80 88". You may need different NLP techniques to accurately extract information out of different sections. Sentence splitting, for example, may be different in subjective versus objective sections. Radiology reports are not in the SOAP format, but section identification can still be very important, because in these reports the radiologist describes what she sees on the radiographic image then sometimes summarizes her impression of what the observations may represent, such as "the infiltrate could be consistent with pneumonia." For some tasks, you may only be interested in the impression, which has more weight than the reasoning that occurs in the Findings section.

## Exercise

Write a function that returns the impression section of a report. The function will take two arguments:

1. The text to search for the impression section is the first positional argument.
1. A list of phrases to use to identify the impression section.

Not every report will have a section labeled "IMPRESSION". You can consider "INTERPRETATION" and "CONCLUSIONS" as synonyms for "IMPRESSION"

If you cannot identify the impresison section (or equivalent), return an empty string.

#### Hints

* Use a for loop  to iterate across
* Use the find method of a string to identify where in the string the impression section occurs
* Use slicing to grab the impression section


In [27]:

def find_impression(txt, searchterms=["IMPRESSION","INTERPRETATION","CONCLUSIONS"]):
    #for x in range(len(rad_Data["text"])):
    for term in searchterms:
        if rad_Data["text"][txt].find(term)!=-1
            return(rad_Data["text"][txt][rad_Data["text"][x].find(term):])
            break
    return ""

find_impression(txt10)


SyntaxError: invalid syntax (<ipython-input-27-0979e293500c>, line 5)

In [22]:
rad_Data["text"][0][25:]

'**2644-1-17**]              Discharge Date:   [**2644-1-23**]\n \nDate of Birth:  [**2553-5-26**]             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nPatient recorded as having No Known Allergies to Drugs\n \nAttending:[**First Name3 (LF) 2775**] \nChief Complaint:\nfall\n \nMajor Surgical or Invasive Procedure:\nn/a\n\n \nHistory of Present Illness:\nNF admit seen and appreciated.  Briefly this is a [**Age over 90 **] yo f w/ h/o \nlung ca w/ metastasis to brain on XRT, s/p fall at NH on Sun \neve. No LOC.  [**1-24**] lethargy, n/v,L arm weakness, sent to [**Hospital1 **] \n[**Location (un) 579**] where she was noted to havv 3x3cm R posterior parietal \nbleed.  Tx to NSICU, started on dilantin (load) and decadron.  \nDeemed not a surgical candidate -> DNR/DNI.\nNoted to be guiac pos, +hct drop, coffee grounds on NGL.  GI \nrecommended transfusing to maintain hct.\n \nPast Medical History:\nLung ca w/ mets to brain\nhigh cholesterol\nHTN\nCAD\nLopresser 25mg tid\n \nSocial His

## Exercise 

Use ``apply()`` to create a new column in the ``rad_data`` named "IMPRESSIONS" with the string identified with ``find_impression()``.

In [ ]:
rad_data.head()

## How else might we need to break up a report?

* How would I break a report into sentences?
* How would I break a report into words?

## Group Exercise

### Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame
* Use TextBlob to get words